# Settings

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
images_dir = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
validation_percentage = 20
valid_format = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

In [6]:
os.getcwd()

'C:\\Users\\Khalid\\Documents\\projects\\Generatives\\Generative_Models'

In [7]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [8]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [9]:
inputs_shape= image_size=(200, 200, 3)
batch_size = 32
latents_dim = 32
intermediate_dim = 16

In [10]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [11]:
import tensorflow as tf

In [12]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

In [13]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [14]:
_instance_scale

1.0

In [15]:
import numpy as np
from collections.abc import Iterable

In [16]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [17]:
_outputs_shape

120000

# Model's Layers definition

In [18]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [19]:
model_name = dataset_name+'ITAAEpsnr'
experiments_dir='experiments'+sep_local+model_name

In [20]:
experiments_dir

'experiments\\pokemonIAAEpsnr'

In [21]:
from training.adversarial_basic.generative_adversarial.autoencoders.AAE import AAE as AE

In [22]:
inputs_shape=image_size

In [23]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [24]:
from utils.data_and_files.file_utils import create_if_not_exist

In [25]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [26]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonIAAEpsnr\\var_save_dir'

In [27]:
#to restore trained model, set filepath=_restore

In [ ]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses, \

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'generative_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [28]:
ae = AE( 
    name= model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 16)      64        
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 120000)            2040000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [29]:
from evaluation.quantitive_metrics.peak_signal_to_noise_ratio import prepare_psnr
from statistical.losses_utilities import similarty_to_distance


In [30]:
ae.compile(loss={'x_logits': similarity_to_distance(prepare_psnr([ae.batch_size]+ae.get_inputs_shape()))})

Model: "pokemonIAAEpsnr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                20480224  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       2041596   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 22,521,820
Trainable params: 22,521,750
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [31]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [33]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [34]:
_restore

'experiments\\pokemonIAAEpsnr\\var_save_dir'

In [35]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [36]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonIAAEpsnr\\csv_dir\\pokemonIAAEpsnr.csv'

In [37]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [38]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [39]:
import numpy as np

# Model Training

In [40]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)


training traditional basicAE
Train for 10 steps, validate for 10 steps


Epoch 1/10
10/10 [==============================] - ETA: 31s - loss: 7.3825 - prepare_psnr(inputs_flat_shape): 7.3397 - prepare_ssim_multiscale(inputs_flat_shape): 0.0342 - sharp_diff: 7.311 - ETA: 14s - loss: 7.3511 - prepare_psnr(inputs_flat_shape): 7.3056 - prepare_ssim_multiscale(inputs_flat_shape): 0.0320 - sharp_diff: 7.192 - ETA: 8s - loss: 7.3185 - prepare_psnr(inputs_flat_shape): 7.2680 - prepare_ssim_multiscale(inputs_flat_shape): 0.0311 - sharp_diff: 7.114 - ETA: 5s - loss: 7.2827 - prepare_psnr(inputs_flat_shape): 7.2283 - prepare_ssim_multiscale(inputs_flat_shape): 0.0309 - sharp_diff: 7.07 - ETA: 4s - loss: 7.2614 - prepare_psnr(inputs_flat_shape): 7.2041 - prepare_ssim_multiscale(inputs_flat_shape): 0.0316 - sharp_diff: 7.02 - ETA: 2s - loss: 7.2539 - prepare_psnr(inputs_flat_shape): 7.1918 - prepare_ssim_multiscale(inputs_flat_shape): 0.0320 - sharp_diff: 6.98 - ETA: 1s - loss: 7.2365 - prepare_psnr(inputs_flat_shape): 7.1711 - prepare_ssim_multiscale(inputs_flat_shape)

Epoch 2/10
10/10 [==============================] - ETA: 1s - loss: 7.0770 - prepare_psnr(inputs_flat_shape): 6.9772 - prepare_ssim_multiscale(inputs_flat_shape): 0.0185 - sharp_diff: 5.87 - ETA: 1s - loss: 7.0058 - prepare_psnr(inputs_flat_shape): 6.9050 - prepare_ssim_multiscale(inputs_flat_shape): 0.0187 - sharp_diff: 5.84 - ETA: 1s - loss: 6.9851 - prepare_psnr(inputs_flat_shape): 6.8847 - prepare_ssim_multiscale(inputs_flat_shape): 0.0193 - sharp_diff: 5.89 - ETA: 0s - loss: 6.9857 - prepare_psnr(inputs_flat_shape): 6.8848 - prepare_ssim_multiscale(inputs_flat_shape): 0.0200 - sharp_diff: 5.90 - ETA: 0s - loss: 6.9532 - prepare_psnr(inputs_flat_shape): 6.8514 - prepare_ssim_multiscale(inputs_flat_shape): 0.0199 - sharp_diff: 5.92 - ETA: 0s - loss: 6.9501 - prepare_psnr(inputs_flat_shape): 6.8473 - prepare_ssim_multiscale(inputs_flat_shape): 0.0213 - sharp_diff: 5.90 - ETA: 0s - loss: 6.9270 - prepare_psnr(inputs_flat_shape): 6.8230 - prepare_ssim_multiscale(inputs_flat_shape): 0.0

Epoch 3/10
10/10 [==============================] - ETA: 1s - loss: 6.6904 - prepare_psnr(inputs_flat_shape): 6.5700 - prepare_ssim_multiscale(inputs_flat_shape): 0.0192 - sharp_diff: 5.48 - ETA: 1s - loss: 6.7392 - prepare_psnr(inputs_flat_shape): 6.6190 - prepare_ssim_multiscale(inputs_flat_shape): 0.0204 - sharp_diff: 5.62 - ETA: 1s - loss: 6.5724 - prepare_psnr(inputs_flat_shape): 6.4512 - prepare_ssim_multiscale(inputs_flat_shape): 0.0185 - sharp_diff: 5.42 - ETA: 0s - loss: 6.6236 - prepare_psnr(inputs_flat_shape): 6.5009 - prepare_ssim_multiscale(inputs_flat_shape): 0.0189 - sharp_diff: 5.48 - ETA: 0s - loss: 6.5775 - prepare_psnr(inputs_flat_shape): 6.4532 - prepare_ssim_multiscale(inputs_flat_shape): 0.0186 - sharp_diff: 5.48 - ETA: 0s - loss: 6.5719 - prepare_psnr(inputs_flat_shape): 6.4458 - prepare_ssim_multiscale(inputs_flat_shape): 0.0186 - sharp_diff: 5.47 - ETA: 0s - loss: 6.5928 - prepare_psnr(inputs_flat_shape): 6.4662 - prepare_ssim_multiscale(inputs_flat_shape): 0.0

Epoch 4/10
10/10 [==============================] - ETA: 1s - loss: 6.2838 - prepare_psnr(inputs_flat_shape): 6.1303 - prepare_ssim_multiscale(inputs_flat_shape): 0.0164 - sharp_diff: 5.28 - ETA: 1s - loss: 6.2808 - prepare_psnr(inputs_flat_shape): 6.1303 - prepare_ssim_multiscale(inputs_flat_shape): 0.0158 - sharp_diff: 5.34 - ETA: 1s - loss: 6.2907 - prepare_psnr(inputs_flat_shape): 6.1392 - prepare_ssim_multiscale(inputs_flat_shape): 0.0159 - sharp_diff: 5.41 - ETA: 0s - loss: 6.2433 - prepare_psnr(inputs_flat_shape): 6.0902 - prepare_ssim_multiscale(inputs_flat_shape): 0.0151 - sharp_diff: 5.40 - ETA: 0s - loss: 6.2231 - prepare_psnr(inputs_flat_shape): 6.0680 - prepare_ssim_multiscale(inputs_flat_shape): 0.0156 - sharp_diff: 5.43 - ETA: 0s - loss: 6.1893 - prepare_psnr(inputs_flat_shape): 6.0324 - prepare_ssim_multiscale(inputs_flat_shape): 0.0155 - sharp_diff: 5.45 - ETA: 0s - loss: 6.1911 - prepare_psnr(inputs_flat_shape): 6.0333 - prepare_ssim_multiscale(inputs_flat_shape): 0.0

...

Epoch 6/10
 9/10 [==========================>...] - ETA: 2s - loss: 46819.0625 - generative_discriminator_fake_outputs_loss: 0.1976 - generative_discriminator_fake_outputs_loss: 0.3093 - x_logits_loss: 46818.3711 - x_logits_psnr: 8.7802 - x_logits_ssmi: 0.0428 - x_logits_sharp_diff: 7.198 - ETA: 2s - loss: 47113.2305 - generative_discriminator_fake_outputs_loss: 0.2005 - generative_discriminator_fake_outputs_loss: 0.3180 - x_logits_loss: 47112.5312 - x_logits_psnr: 8.7599 - x_logits_ssmi: 0.0408 - x_logits_sharp_diff: 7.262 - ETA: 2s - loss: 48355.9831 - generative_discriminator_fake_outputs_loss: 0.1963 - generative_discriminator_fake_outputs_loss: 0.3324 - x_logits_loss: 48355.2695 - x_logits_psnr: 8.6653 - x_logits_ssmi: 0.0386 - x_logits_sharp_diff: 7.284 - ETA: 1s - loss: 49150.8076 - generative_discriminator_fake_outputs_loss: 0.1916 - generative_discriminator_fake_outputs_loss: 0.3437 - x_logits_loss: 49150.0859 - x_logits_psnr: 8.5927 - x_logits_ssmi: 0.0372 - x_logits_sharp_di

Epoch 7/10
10/10 [==============================] - ETA: 3s - loss: 48206.1719 - generative_discriminator_fake_outputs_loss: 0.1850 - generative_discriminator_fake_outputs_loss: 0.3143 - x_logits_loss: 48205.4844 - x_logits_psnr: 8.6921 - x_logits_ssmi: 0.0396 - x_logits_sharp_diff: 7.356 - ETA: 2s - loss: 47353.4043 - generative_discriminator_fake_outputs_loss: 0.1894 - generative_discriminator_fake_outputs_loss: 0.3000 - x_logits_loss: 47352.7266 - x_logits_psnr: 8.7796 - x_logits_ssmi: 0.0407 - x_logits_sharp_diff: 7.284 - ETA: 2s - loss: 46717.9674 - generative_discriminator_fake_outputs_loss: 0.1859 - generative_discriminator_fake_outputs_loss: 0.2916 - x_logits_loss: 46717.3008 - x_logits_psnr: 8.8361 - x_logits_ssmi: 0.0419 - x_logits_sharp_diff: 7.275 - ETA: 2s - loss: 47609.2949 - generative_discriminator_fake_outputs_loss: 0.1835 - generative_discriminator_fake_outputs_loss: 0.2921 - x_logits_loss: 47608.6328 - x_logits_psnr: 8.7460 - x_logits_ssmi: 0.0396 - x_logits_sharp_di

Epoch 8/10
10/10 [==============================] - ETA: 2s - loss: 60465.1445 - generative_discriminator_fake_outputs_loss: 0.1703 - generative_discriminator_fake_outputs_loss: 0.2840 - x_logits_loss: 60464.5078 - x_logits_psnr: 7.7133 - x_logits_ssmi: 0.0154 - x_logits_sharp_diff: 7.361 - ETA: 2s - loss: 54691.2754 - generative_discriminator_fake_outputs_loss: 0.1719 - generative_discriminator_fake_outputs_loss: 0.2782 - x_logits_loss: 54690.6406 - x_logits_psnr: 8.1655 - x_logits_ssmi: 0.0279 - x_logits_sharp_diff: 7.336 - ETA: 2s - loss: 53134.3477 - generative_discriminator_fake_outputs_loss: 0.1722 - generative_discriminator_fake_outputs_loss: 0.2841 - x_logits_loss: 53133.7070 - x_logits_psnr: 8.2979 - x_logits_ssmi: 0.0298 - x_logits_sharp_diff: 7.323 - ETA: 2s - loss: 51547.3516 - generative_discriminator_fake_outputs_loss: 0.1733 - generative_discriminator_fake_outputs_loss: 0.2833 - x_logits_loss: 51546.7109 - x_logits_psnr: 8.4366 - x_logits_ssmi: 0.0314 - x_logits_sharp_di

Epoch 9/10
10/10 [==============================] - ETA: 3s - loss: 46452.2695 - generative_discriminator_fake_outputs_loss: 0.1797 - generative_discriminator_fake_outputs_loss: 0.2707 - x_logits_loss: 46451.6406 - x_logits_psnr: 8.8863 - x_logits_ssmi: 0.0398 - x_logits_sharp_diff: 7.238 - ETA: 2s - loss: 46484.1953 - generative_discriminator_fake_outputs_loss: 0.1720 - generative_discriminator_fake_outputs_loss: 0.2815 - x_logits_loss: 46483.5625 - x_logits_psnr: 8.8759 - x_logits_ssmi: 0.0405 - x_logits_sharp_diff: 7.200 - ETA: 2s - loss: 46769.5182 - generative_discriminator_fake_outputs_loss: 0.1684 - generative_discriminator_fake_outputs_loss: 0.2805 - x_logits_loss: 46768.8867 - x_logits_psnr: 8.8492 - x_logits_ssmi: 0.0416 - x_logits_sharp_diff: 7.215 - ETA: 2s - loss: 46978.4326 - generative_discriminator_fake_outputs_loss: 0.1671 - generative_discriminator_fake_outputs_loss: 0.2837 - x_logits_loss: 46977.7969 - x_logits_psnr: 8.8316 - x_logits_ssmi: 0.0421 - x_logits_sharp_di

Epoch 10/10
10/10 [==============================] - ETA: 2s - loss: 47817.8438 - generative_discriminator_fake_outputs_loss: 0.1526 - generative_discriminator_fake_outputs_loss: 0.2315 - x_logits_loss: 47817.2734 - x_logits_psnr: 8.7202 - x_logits_ssmi: 0.0356 - x_logits_sharp_diff: 7.266 - ETA: 2s - loss: 48215.5410 - generative_discriminator_fake_outputs_loss: 0.1525 - generative_discriminator_fake_outputs_loss: 0.2420 - x_logits_loss: 48214.9609 - x_logits_psnr: 8.6872 - x_logits_ssmi: 0.0356 - x_logits_sharp_diff: 7.268 - ETA: 2s - loss: 48617.7161 - generative_discriminator_fake_outputs_loss: 0.1495 - generative_discriminator_fake_outputs_loss: 0.2431 - x_logits_loss: 48617.1367 - x_logits_psnr: 8.6563 - x_logits_ssmi: 0.0354 - x_logits_sharp_diff: 7.261 - ETA: 2s - loss: 47809.8584 - generative_discriminator_fake_outputs_loss: 0.1501 - generative_discriminator_fake_outputs_loss: 0.2504 - x_logits_loss: 47809.2695 - x_logits_psnr: 8.7409 - x_logits_ssmi: 0.0361 - x_logits_sharp_d

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=100)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [73]:
from training.generators.image_generation_testing import interpolate_a_batch

In [74]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)

100%|██████████| 15/15 [00:00<00:00, 19.90it/s]
